# Transportation Simulation using Agent Based approach 

### MESA Agenet Based library 

- all postcode are coming from  the dataset which has population and in-use postcodes
- This version is using Google Map API
- In this version, Agent base simulation developed using MESA library and shows  faster results compare with my MESA free version


In [ ]:
! pip install googlemaps
! pip install DateTime
! pip install gmplot
! pip install bs4
! pip install geopy
! pip install Mesa

In [11]:
import pandas as pd
import random  #import randrange
import datetime as dt
#Googel API code
import googlemaps
import urllib
import numpy as np
import sys
import math
import geopy
import urllib, json
import geopy.distance
import time
import threading


In [23]:

KEY = ''# Google map API key 
folder=''# inset your databse directory 
gmaps = googlemaps.Client(KEY)  # google API key
dataset = pd.read_csv(folder+'postcodes.csv')

# Importing the dataset
dataset = pd.read_csv(folder+'postcodes.csv')#pd.read_csv('postcodes.csv')#
#dataset=  dataset.dropna(subset=['Population'], inplace=True)
dataset = dataset[dataset['Households']>=15]
postcode=list(dataset['Postcode'])
#demografic= pd.read_csv("filename.csv")
sim_date=[2019, 12, 15] #year,month,day
def random_time(start,h_shift,min_var):#H_shift is the hours after zero_time that one's day starts, min_var is minut variation from the nominal start day time, start is date time with this format (2018, 12, 20,8,00)
        date_time = start + dt.timedelta(hours=h_shift,minutes=random.randrange(min_var),seconds=0) #timedelta(weeks=40, days=84, hours=23,minutes=50, seconds=600)  # adds up to 365 days       
        #print (date_time.strftime("%d/%m/%y %H:%M"))
        #print (date_time.strftime("%H:%M:%S"))
        return date_time
    

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Some useful functions: 


In [24]:


def places_finder(where,radius,types):
    #making the url
    geo= gmaps.geocode(str(where))
    latlng=geo[0]['geometry']['bounds']['northeast']
    
    LOCATION = str(latlng['lat']) + "," + str(latlng['lng'])
    RADIUS = radius
    TYPES = types
    MyUrl = ('https://maps.googleapis.com/maps/api/place/nearbysearch/json'
            '?location=%s'
            '&radius=%s'
            '&types=%s'
            '&sensor=false&key=%s') % (LOCATION, RADIUS, TYPES, KEY)
    #grabbing the JSON result
    response = urllib.request.urlopen(MyUrl)
    jsonRaw = response.read()
    jsonData = json.loads(jsonRaw.decode())
    location=jsonData['results'][0]['geometry']['location']
    return  str(location['lat'])+','+str(location['lng'])
   
def snap2rd(loc1,loc2):
    locs=str(loc1[0])+','+str(loc1[1])+'|'+str(loc2[0])+','+str(loc2[1])
    myurl = ('https://roads.googleapis.com/v1/snapToRoads'
                '?path=%s'
                '&key=%s')%(locs,KEY)
    f = urllib.request.urlopen(myurl)
    q=json.loads(f.read().decode())
    return (q['snappedPoints'][0]['location']['latitude'],q['snappedPoints'][0]['location']['longitude'])



def snap22rd(locs): #works with more than two points
    points=''
    for loc in locs:
        points=points + str(loc[0])+','+str(loc[1])+'|'
    points=points[:-1]
    myurl = ('https://roads.googleapis.com/v1/snapToRoads'
                '?path=%s'
                '&key=%s')%(points,KEY)
    f = urllib.request.urlopen(myurl)
    q=json.loads(f.read().decode())
    return (q['snappedPoints'][0]['location']['latitude'],q['snappedPoints'][0]['location']['longitude'])



##### decoding Google map API data


In [25]:
def decode_polyline(polyline_str):
    index, lat, lng = 0, 0, 0
    coordinates = []
    dists=[0]
    changes = {'latitude': 0, 'longitude': 0}

    # Coordinates have variable length when encoded, so just keep
    # track of whether we've hit the end of the string. In each
    # while loop iteration, a single coordinate is decoded.
    while index < len(polyline_str):
        # Gather lat/lon changes, store them in a dictionary to apply them later
        for unit in ['latitude', 'longitude']: 
            shift, result = 0, 0

            while True:
                byte = ord(polyline_str[index]) - 63
                index+=1
                result |= (byte & 0x1f) << shift
                shift += 5
                if not byte >= 0x20:
                    break

            if (result & 1):
                changes[unit] = ~(result >> 1)
            else:
                changes[unit] = (result >> 1)

        lat += changes['latitude']
        lng += changes['longitude']

        coordinates.append((lat / 100000.0, lng / 100000.0))
        if len(coordinates)>1:
            dists.append(geopy.distance.distance(coordinates[-1], coordinates[-2]).m) #returns the distance in 'm' , replace it with 'km' if needed

    return coordinates, dists



#### Define trip's reules for agents

In [26]:
def trips(home_adrs,work_adrs,gym_adrs=None,shop_adrs=None,house_hdcount=1):
        zero_time = dt.datetime(sim_date[0],sim_date[1],sim_date[2],00,00)   

        trip_nodes={'Single_people':[
                        ['HOME','WORK','HOME'],
                        ['HOME','WORK','GYM','HOME'],
                        ['HOME','WORK','SHOP','HOME'],
                        ['HOME','WORK','GYM','SHOP','HOME'],
                        ['HOME','WORK','SHOP','GYM','HOME'],
                        ['HOME','GYM','WORK','HOME'],
                        ['HOME','GYM','WORK','SHOP','HOME']] 
                       }   
#        trip_nodes={'Single_people':[['HOME','WORK','HOME']]}         
        if work_adrs!=None and house_hdcount==1 : 
            steps=random.choice(trip_nodes['Single_people'])
        trips=[]
        for i in range(len(steps)-1):
            trips.append(steps[0+i:2+i])
            # before work activities
            if trips[i]==['HOME','GYM'] : trips[i].append('leave');trips[i].append(random_time(zero_time,6,40))
            elif trips[i][1]=='WORK' :trips[i].append('arrive');trips[i].append(random_time(zero_time,8,45))
            # after work activities : they are timed based on last activity time (trips[-1][-1])
            
            elif trips[i][0]=='WORK' :trips[i].append('leave');trips[i].append(random_time(trips[-2][-1],8,30)) #time spend at work: 8h
            elif trips[i][0]=='GYM'  :trips[i].append('leave');trips[i].append(random_time(trips[-2][-1],1,30))#time spend at gym: 1h
            elif trips[i][0]=='SHOP' :trips[i].append('leave');trips[i].append(random_time(trips[-2][-1],0,50)) 
            else : pass
        
        #assigning actual addresses to the trip lists:
        actual_places = {'HOME':home_adrs, 'WORK':work_adrs, 'GYM':gym_adrs, 'SHOP':shop_adrs}
        for i in range(len(trips)):
            trips[i]=[actual_places.get(n, n) for n in trips[i]]
        # print (trips)
        return trips  
        

In [27]:
#### Define Route Rules for agents

In [28]:
def route(trip_legs):
        routes=[]
        #print (type(trip_legs))
        for leg in trip_legs:
            #leg[0],leg[1] =origin and destination of each leg,leg[2]is departure or arrival, leg[-1] is the time
            if leg[2]=='leave':
                directions_result = gmaps.directions(leg[0], leg[1], mode="driving", departure_time=leg[-1])
            else: directions_result = gmaps.directions(leg[0],leg[1],mode="driving",arrival_time=leg[-1])                                 
            #extracting the key infor ployline points from a direcion_results 
            route_info=directions_result[0]['legs'][0] 
            #key info : start time, start location, total trip disttance, total trip time ,end location, end time
            if leg[2]=='leave':
                start_time=leg[-1];
                end_time=leg[-1] + dt.timedelta(seconds=route_info['duration']['value'])#this calculates the end time if we know the leave time

            else: 
                start_time=leg[-1] - dt.timedelta(seconds=route_info['duration']['value']) #this calculates the start time if we know the arrival time
                end_time=leg[-1]
        
            leg_key_info={'start_time':start_time,
                                'start_location':route_info['start_location'],
                                'distance':route_info['distance']['value'],
                                'duration':route_info['duration']['value'],
                                'end_location':route_info['end_location'],
                                'end_time':end_time
                            }   
            #extracting the ployline points from a route                                                                                                         
            routes.append({'leg_key_info':leg_key_info,'steps':route_info['steps']})
        #the route is a list of trip route/info for each leg, for instance [0] returns 1st leg and will containe below dictionaries: 'leg_key_info' and 'steps'
        # for l in routes:
        #     print ('start->: ', l['leg_key_info']['start_location'], '@',l['leg_key_info']['start_time'])
        #     print ('--->end: ', l['leg_key_info']['end_location'], '@',l['leg_key_info']['end_time'])
        #     print('---------------------------------------------------------------------------------')
        return routes    



In [29]:
class Spinner:
    busy = False
    delay = 0.1
    @staticmethod
    def spinning_cursor():
        while 1: 
            for cursor in '|/-\\': yield cursor
    def __init__(self, delay=None):
        self.spinner_generator = self.spinning_cursor()
        if delay and float(delay): self.delay = delay
    def spinner_task(self):
        while self.busy:
            sys.stdout.write('\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b'+next(self.spinner_generator)+' WORKING...')
            sys.stdout.flush()
            time.sleep(self.delay)
    def start(self):
        self.busy = True
        threading.Thread(target=self.spinner_task).start()
    def stop(self):
        self.busy = False
        time.sleep(self.delay)        
        sys.stdout.write('\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b'+'FINISHED!')



In [30]:
def init_list_of_objects(size):
    list_of_objects = list()
    for i in range(0,size):
        list_of_objects.append( list() ) #different object reference each time
    return list_of_objects

In [31]:
  
from mesa.space import ContinuousSpace
from mesa import Agent, Model
from mesa.time import SimultaneousActivation


In [32]:
#### define city boarder dimensions
x_min,x_max=-8.7,1.9
y_min,y_max=47.0,62.0

#### Define MESA main classes

In [33]:
class CityModel(Model):
    """A model with some number of agents."""
    def __init__(self, N):
        
        self.num_agents = N
        self.AreaMap = ContinuousSpace(x_max, y_max, True, x_min, y_min)
        self.schedule = SimultaneousActivation(self)
        # Create agents
        for i in range(self.num_agents):
            #randon postcode for HOME and WORK

            h=random.choice(postcode)#sample_home_postcode)
            w=random.choice(postcode)#sample_work_postcode)
            
            a = CityAgent(i, h,'gender',32,'white_collar',1,w,self)
            self.schedule.add(a)

            x = gmaps.geocode(h)[0]['geometry']['bounds']['northeast']['lng']
            y = gmaps.geocode(h)[0]['geometry']['bounds']['northeast']['lat']
            self.AreaMap.place_agent(a, (x, y))
            sys.stdout.write('\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b'+'Agent: '+str(1+i)+' / '+str(N))
            sys.stdout.flush()
    def step(self):
        self.schedule.step()
        


In [34]:
class CityAgent(Agent):
    """ An agent with fixed initial location."""
    def __init__(self, unique_id, home_adrs,gender,age,social_cls,house_hdcount,work_adrs,model):#cars=None,job_cls=None,n_child=None,child_ages=[], model):
        super().__init__(unique_id, model)
        self.id=unique_id
        self.home_adrs=home_adrs
        self.gender=gender
        self.age=age
        self.social_cls=social_cls
        self.house_hdcount=house_hdcount
#        self.cars=cars
        self.work_adrs=work_adrs
#        self.job_cls=job_cls
#        self.n_child=n_child
#        self.child_ages=child_ages
        
        # additional places:
        self.gym_adrs= (51.764156, -1.258917)#places_finder(self.home_adrs,5000,'gym')
        self.nursery_adrs=(51.764156, -1.258917)# places_finder(self.home_adrs,5000,'nursery')
        self.shop_adrs= (51.764156, -1.258917)#places_finder(self.home_adrs,5000,'supermarket')

        # Agent's trips and associated routes:
        self.trips=trips(self.home_adrs,self.work_adrs,self.gym_adrs,self.shop_adrs,self.house_hdcount)
        self.route=route(self.trips)
        
    def loc8(self,t): #location of agent at time = t (datetime.datetime type)

        for l in self.route:
            if  t <= l['leg_key_info']['start_time']: 
                loc=(l['leg_key_info']['start_location']['lat'],l['leg_key_info']['start_location']['lng'])
                return loc #(loc[1],loc[0])
                break
                

            elif l['leg_key_info']['start_time'] < t <= l['leg_key_info']['end_time']:
                t=t-l['leg_key_info']['start_time']
                t=int(t.total_seconds())
                i=0
                while t>l['steps'][i]['duration']['value']:
                    t=t-l['steps'][i]['duration']['value']
                    i+=1 #this 'i' once out of the shows in which step of the leg the agent is 
                #making a displacement vector from start position (ps) and end position (pe)

                step_points,step_length=decode_polyline(l['steps'][i]['polyline']['points'])#**************************
                step_velocity=l['steps'][i]['distance']['value']/l['steps'][i]['duration']['value']
                step_time=np.array(step_length)/step_velocity # this list 'step_time' starts with '0' as first item
                ii=0
#                while ii+1<=len(step_time) and t>step_time[ii+1]:
#                    t=t-step_time[ii+1]
#                    ii+=1
                while ii+1<len(step_time) and t>step_time[ii+1]:
                    t=t-step_time[ii+1]
                    ii+=1
                try: pe=step_points[ii+1];ps=step_points[ii];tt=t/step_time[ii+1]
                except:pe=step_points[-1];ps=step_points[-2];tt=t/step_time[-1]
                
                vec=np.array(pe)-np.array(ps)
                #making a partial displacement vector based on the time passed from start position
                vec=tt*vec
                #current position : ps + partial vector
                loc=tuple(ps+vec)# array
                return loc
                break
            else: 
                loc=(l['leg_key_info']['end_location']['lat'],l['leg_key_info']['end_location']['lng']);
                return loc 
                
       
    def move(self):

        new_position = self.loc8(t)
        #print ('time:',t,'  new',new_position)
        self.model.AreaMap.move_agent(self, new_position)
        LOCs[time_list.index(t)].append(new_position)




    def step(self):
        self.move()
        sys.stdout.write('\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b'+"@ "+str(t))
        sys.stdout.flush()


    def advance(self):
        pass


## Simulation

In [40]:
population=50

sim_date=[2019, 12, 20] #year,month,day DATE of simulation
t_start=dt.datetime(sim_date[0],sim_date[1],sim_date[2], 7,50)# start time of simulation
t_end=dt.datetime(sim_date[0],sim_date[1],sim_date[2], 8,10)# end time of simulation
time_resolution=9*60 #seconds
#LOCs=[[]]*int((t_end-t_start).total_seconds()/time_resolution)
nn=int(math.ceil((t_end-t_start).total_seconds()/time_resolution))
LOCs=init_list_of_objects(nn)
time_list=[]

#spinner = Spinner()
#spinner.start()
time1=dt.datetime.now()
print('Creating Agents:')
model = CityModel(population)
#print(dt.datetime.now())
t=t_start
#model.step()
print ("\nlocating agents at various moments:")
while t<t_end :
     time_list.append(t)
     model.step()
     t = t + dt.timedelta(seconds=time_resolution)
    
#spinner.stop()
#print (dt.datetime.now()-time1)





Creating Agents:
Agent: 1 / 5Agent: 2 / 5Agent: 3 / 5Agent: 4 / 5Agent: 5 / 5Agent: 6 / 5Agent: 7 / 5Agent: 8 / 5Agent: 9 / 5Agent: 10 / 5Agent: 11 / 5Agent: 12 / 5Agent: 13 / 5Agent: 14 / 5Agent: 15 / 5Agent: 16 / 5Agent: 17 / 5Agent: 18 / 5Agent: 19 / 5Agent: 20 / 5Agent: 21 / 5Agent: 22 / 5Agent: 23 / 5Agent: 24 / 5Agent: 25 / 5Agent: 26 / 5Agent: 27 / 5Agent: 28 / 5Agent: 29 / 5Agent: 30 / 5Agent: 31 / 5Agent: 32 / 5Agent: 33 / 5Agent: 34 / 5Agent: 35 / 5Agent: 36 / 5Agent: 37 / 5Agent: 38 / 5Agent: 39 / 5Agent: 40 / 5Agent: 41 / 5Agent: 42 / 5Agent: 43 / 5Agent: 44 / 5Agent: 45 / 5Agent: 46 / 5Agent: 47 / 5Agent: 48 / 5Agent: 49 / 5Agent: 50 / 50
locating agents at various moments:
@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50:@ 2019-12-20 07:50

## Visualization

In [42]:
# Place map
from gmplot import gmplot
import os
import time

#https://media.readthedocs.org/pdf/jupyter-gmaps/latest/jupyter-gmaps.pdf
#https://github.com/vgm64/gmplot/issues/16
from bs4 import BeautifulSoup
def insertapikey(fname, apikey):
    """put the google api key in a html file"""
    def putkey(htmltxt, apikey, apistring=None):
        """put the apikey in the htmltxt and return soup"""
        if not apistring:
            apistring = "https://maps.googleapis.com/maps/api/js?key=%s&callback=initMap"
        soup = BeautifulSoup(htmltxt, 'html.parser')
        body = soup.body
        src = apistring % (apikey, )
        tscript = soup.new_tag("script", src=src)#, async="defer")
        body.insert(-1, tscript)
        return soup
    htmltxt = open(fname, 'r').read()
    soup = putkey(htmltxt, apikey)
    newtxt = soup.prettify()
    open(fname, 'w').write(newtxt)



#http://www.googlemapsmarkers.com/v1/'1'/'cornflowerblue'/ #http://www.googlemapsmarkers.com/

print ("\nCreating the HTML file:")
for i in range(len(time_list)):
    gmap = gmplot.GoogleMapPlotter(51.827922, -0.790973, 11)
    p_lat,p_lng= zip(*LOCs[i])
    #gmap.marker(51.827922, -0.790973, 'cornflowerblue')
    gmap.scatter(p_lat, p_lng, 'red', size=200, marker=False)
    # Draw
    #gmap.plot(p_lat, p_lng, 'red')
    f=str(time_list[i]).replace(":","-")
    f=f.replace(" ","_")#just to get rid of un-acceptable charatcters
    map_file=folder+f+".html" #spide mac and windows 

    gmap.draw(map_file)
    insertapikey(map_file, KEY)
    sys.stdout.write('\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b'+str(f+'.html'))
    sys.stdout.flush()    
    
    try:os.system("open "+map_file)  
    except: pass #OSX
    try:os.system("start "+map_file) 
    except: pass#Windows 

    


Creating the HTML file:
2019-12-20_07-50-00.ht2019-12-20_07-59-00.ht2019-12-20_08-08-00.html